# 01. YOLO를 활용한 실시간 객체 탐지 (Detection & Classification)

## 01-1. YOLO 개념 및 다양한 활용 방식

### YOLO란?
- YOLO(You Only Look Once)는 단일 신경망을 사용하여 한 번의 연산으로 객체의 위치와 클래스 정보를 예측하는 객체 탐지 모델입니다.
- 기존의 R-CNN 및 Faster R-CNN과 같은 모델은 후보 영역을 먼저 찾고 이를 분류하는 방식이지만, YOLO는 전체 이미지를 한 번에 분석하여 예측을 수행하므로 속도가 빠릅니다.

### YOLO의 작동 방식
- YOLO는 CNN(Convolutional Neural Network) 기반으로 동작하며, 입력 이미지를 특징 맵으로 변환한 후 여러 개의 격자로 나누어 각 격자가 객체의 바운딩 박스와 클래스 확률을 예측합니다.
- 기존 R-CNN 기반 탐지 모델보다 속도가 빠르고, 실시간 처리에 적합합니다.

### YOLO 버전별 특징 및 차이점
- **YOLOv1~v3**: 초기 YOLO 모델로, 속도가 빠르지만 정확도가 상대적으로 낮았습니다.
- **YOLOv4**: CSPDarknet53 백본을 도입하여 정확도를 향상시킨 버전입니다.
- **YOLOv5**: PyTorch 기반으로 구현되었으며, 경량화된 모델(Tiny, Nano 등)을 포함하여 다양한 환경에서 활용할 수 있도록 최적화되었습니다.
- **YOLOv6~v8**: 최신 YOLO 버전으로, 속도와 정확도를 균형 있게 개선하며, Segmentation 및 Classification 기능도 지원합니다.

### Tiny 모델과 Nano 모델의 차이점 및 선택 이유
- **YOLO Tiny**: YOLO의 경량화된 버전으로, 연산량을 줄여 빠른 속도를 제공하지만 정확도는 낮을 수 있습니다.
- **YOLO Nano**: YOLOv5 및 v8에서 등장한 초경량 모델로, 경량화와 성능의 균형을 맞추며, Raspberry Pi와 같은 엣지 디바이스에서도 원활하게 동작할 수 있도록 최적화되었습니다.
- **Raspberry Pi 강의에서 사용할 모델**: Raspberry Pi의 연산 능력을 고려하여 YOLOv5 Nano 모델을 선택하며, 이를 TFLite로 변환하여 경량화된 형태로 실행할 것입니다.

# 02. [실습 6] YOLOv5-Nano 모델 변환 및 실행

## 02-1. Raspberry Pi에서 실행 시 필요한 설정 및 환경 구축
```bash
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip install -r requirements.txt
```

## 02-2. YOLOv5 Nano 모델을 TFLite로 변환
### YOLOv5n 모델 다운로드 및 변환

```bash
python3 export.py --weights yolov5n.pt --include tflite
```

### 모델 확인

In [34]:
import numpy as np
import tensorflow as tf

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="yolov5/yolov5n-fp16.tflite")
interpreter.allocate_tensors()

# 입력/출력 정보 확인
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("입력 정보:", input_details)
print("출력 정보:", output_details)

# 랜덤 입력 데이터 생성 (batch=1, height=640, width=640, channel=3) - 올바른 형식
input_data = np.random.rand(1, 640, 640, 3).astype(np.float32)  # (1, 640, 640, 3)

# 입력 데이터 설정 및 실행
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# 결과 가져오기
output_data = interpreter.get_tensor(output_details[0]['index'])
print("출력 데이터:", output_data.shape)


입력 정보: [{'name': 'serving_default_keras_tensor_121:0', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
출력 정보: [{'name': 'StatefulPartitionedCall_1:0', 'index': 530, 'shape': array([    1, 25200,    85], dtype=int32), 'shape_signature': array([    1, 25200,    85], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
출력 데이터: (1, 25200, 85)


## 02-3. 실시간 카메라 스트리밍과 연결하여 객체 탐지

### 라이브러리 및 환경 설정

In [35]:
import cv2
import numpy as np
import tensorflow as tf

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="yolov5/yolov5n-fp16.tflite")
interpreter.allocate_tensors()

# 입력 / 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# YOLOv5 Nano 모델 정보
INPUT_WIDTH = 640
INPUT_HEIGHT = 640

# 클래스 이름 (YOLOv5 기본 COCO 데이터셋 기준)
COCO_CLASSES = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
    "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier",
    "toothbrush"
]

### 객체 탐지 함수 정의

In [44]:
# 객체 탐지 함수 (TFLite)
def detect_objects(image, conf_threshold=0.9, iou_threshold=0.6):
    # 원본 이미지 크기 저장
    orig_height, orig_width, _ = image.shape

    # 이미지 전처리 (TFLite 형식에 맞게 변환)
    input_image = cv2.resize(image, (INPUT_WIDTH, INPUT_HEIGHT))
    input_image = input_image.astype(np.float32) / 255.0  # 정규화
    input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가

    # 모델 입력 설정
    interpreter.set_tensor(input_details[0]['index'], input_image)

    # 추론 실행
    interpreter.invoke()

    # YOLOv5 TFLite 단일 출력 처리
    output_data = interpreter.get_tensor(output_details[0]['index'])  # (1, 25200, 85)
    output_data = np.squeeze(output_data)  # (25200, 85)로 변환

    # 바운딩 박스 및 신뢰도 추출
    boxes = output_data[:, :4]  # x_center, y_center, w, h
    confidences = output_data[:, 4]  # confidence score
    class_probs = output_data[:, 5:]  # 클래스 확률값 (80개)

    # 클래스 및 점수 추출
    class_ids = np.argmax(class_probs, axis=1)  # 가장 높은 확률의 클래스 선택
    scores = np.max(class_probs, axis=1)  # 해당 클래스의 신뢰도 점수

    # 신뢰도 기준 적용
    indices = np.where(scores > conf_threshold)[0]  # 신뢰도 0.5 이상 객체만 선택
    boxes, scores, class_ids = boxes[indices], scores[indices], class_ids[indices]

    # 바운딩 박스 좌표 변환 (x_center, y_center, w, h → x1, y1, x2, y2)
    scaled_boxes = []
    for i in range(len(boxes)):
        x_center, y_center, w, h = boxes[i]
        x1 = int((x_center - w / 2) * orig_width)
        y1 = int((y_center - h / 2) * orig_height)
        x2 = int((x_center + w / 2) * orig_width)
        y2 = int((y_center + h / 2) * orig_height)
        scaled_boxes.append([x1, y1, x2, y2])

    # Non-Maximum Suppression (NMS) 적용하여 중복 박스 제거
    indices = cv2.dnn.NMSBoxes(scaled_boxes, scores.tolist(), conf_threshold, iou_threshold)
    
    detected_objects = []
    if len(indices) > 0:
        for i in indices.flatten():
            class_name = COCO_CLASSES[class_ids[i]] if class_ids[i] < len(COCO_CLASSES) else "Unknown"
            detected_objects.append((*scaled_boxes[i], scores[i], class_name))

    return detected_objects

### 웹캠에서 객체 탐지 실행

In [45]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 실행
    objects = detect_objects(frame)

    # 탐지된 객체를 화면에 표시
    for (x1, y1, x2, y2, confidence, class_name) in objects:
        label = f"{class_name}: {confidence:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 화면에 출력
    cv2.imshow("YOLOv5n TFLite Object Detection", frame)

    # q 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 03. YOLO를 활용한 이미지 분류(Classification)

## 03-1. YOLO를 이미지 분류용으로 활용하는 방법
- YOLO는 기본적으로 객체 탐지 모델이지만, 분류(Classification) 전용 구조로 활용할 수 있습니다.
- YOLO의 분류 모델은 기존 CNN 기반 분류 모델(ResNet, VGG 등)과 달리, 경량화되어 빠른 속도를 제공합니다.
- YOLO는 Backbone(특징 추출 네트워크)을 활용하여 분류 태스크를 수행하며, 고해상도 입력 이미지에 대해 빠르고 정확한 예측이 가능합니다.
- YOLOv5 및 YOLOv8에서는 Classification 전용 학습 기능이 제공되며, 사전 학습된 모델을 통해 특정 데이터셋에 맞춰 미세 조정(Fine-Tuning)할 수도 있습니다.
- YOLO의 분류 모델은 단순한 이미지 분류뿐만 아니라, 산업용 검사, 의료 영상 분석, 품질 관리 등 다양한 실전 응용 분야에서 활용될 수 있습니다.
- YOLO는 경량화된 구조 덕분에 엣지 디바이스(Raspberry Pi, Jetson Nano 등)에서도 원활하게 동작할 수 있으며, 실시간 이미지 분류를 가능하게 합니다.

## 03-2. YOLO 기반 분류 모델의 활용 사례
- **의료 영상 분석**: X-ray 또는 MRI 이미지를 분석하여 질병을 분류하는 데 활용
- **제조업 품질 관리**: 제품의 불량 여부를 실시간으로 감지하여 공정 자동화 가능
- **소매 및 보안 시스템**: 얼굴 인식을 통한 신원 확인 및 고객 행동 분석
- **일반적인 이미지 분류**: 개, 고양이 분류와 같은 일반적인 이미지 분류 태스크 적용

---

# 04. [실습 7] YOLO를 활용한 이미지 분류

## 04-1. 사전 학습된 YOLO 분류 모델 다운로드 및 실행

### Ultralytics을 활용한 tflite 모델 다운로드
이전 실습 방식에선 직접 git을 불러와 convert를 진행했지만, YOLOv8 버전부터는 Ultralytics 라이브러리활용하여 간단히 변환할 수 있습니다.

In [5]:
import torch
import tensorflow as tf
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')  # 사전 학습된 YOLOv8 분류 모델 로드
model.export(format='tflite')  # tflite로 변환

Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.6.0+cu124 CPU (12th Gen Intel Core(TM) i5-12400F)
YOLOv8n-cls summary (fused): 73 layers, 2,715,880 parameters, 0 gradients, 4.3 GFLOPs

PyTorch: starting from 'yolov8n-cls.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 1000) (5.3 MB)

TensorFlow SavedModel: starting export with tensorflow 2.18.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 0.2s, saved as 'yolov8n-cls.onnx' (10.4 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.26.3...


I0000 00:00:1739403119.766232    5193 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1739403119.766300    5193 single_machine.cc:361] Starting new session
W0000 00:00:1739403119.917994    5193 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1739403119.918011    5193 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-13 08:32:00.035890: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3893] Estimated count of arithmetic ops: 477.397 M  ops, equivalently 238.698 M  MACs
I0000 00:00:1739403120.148118    5193 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1739403120.148187    5193 single_machine.cc:361] Starting new session
W0000 00:00:1739403120.297677    5193 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1739403120.297692    5193 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-13 08:32:00

TensorFlow SavedModel: export success ✅ 8.8s, saved as 'yolov8n-cls_saved_model' (26.2 MB)

TensorFlow Lite: starting export with tensorflow 2.18.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8n-cls_saved_model/yolov8n-cls_float32.tflite' (10.4 MB)

Export complete (9.0s)
Results saved to /home/paymentinapp/raspberrypi/04_YOLO를_활용한_실시간_객체_탐지
Predict:         yolo predict task=classify model=yolov8n-cls_saved_model/yolov8n-cls_float32.tflite imgsz=224  
Validate:        yolo val task=classify model=yolov8n-cls_saved_model/yolov8n-cls_float32.tflite imgsz=224 data=../datasets/imagenet  
Visualize:       https://netron.app


'yolov8n-cls_saved_model/yolov8n-cls_float32.tflite'

## 04-2. 라즈베리파이에서 실시간 카메라를 활용한 이미지 분류

### 실시간으로 YOLO 분류 모델

In [8]:
import cv2
import numpy as np
import tensorflow as tf

# TFLite 모델 로드
interpreter = tf.lite.Interpreter(model_path="./yolov8n-cls_saved_model/yolov8n-cls_float32.tflite")
interpreter.allocate_tensors()

# 입력 및 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 모델이 기대하는 입력 크기 확인
input_shape = input_details[0]['shape']  # (1, 224, 224, 3)
input_size = (input_shape[1], input_shape[2])

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 이미지 크기 변환 (모델의 기대 크기: 224x224)
    resized_frame = cv2.resize(frame, input_size)
    input_data = np.expand_dims(resized_frame, axis=0).astype(np.float32)  # 차원 확장 및 float32 변환

    # TFLite 모델 실행
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])

    # 가장 높은 확률의 클래스 가져오기
    predicted_class = np.argmax(predictions)

    # 화면에 분류 결과 표시
    cv2.putText(frame, f'Class: {predicted_class}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('YOLO Classification (TFLite)', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
